In [7]:
import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, UpSampling2D
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils

import numpy as np
import os
import requests

In [8]:
print(tf.__version__)

2.0.0


In [9]:
print(os.getcwd())



C:\Users\Kittiphong\sidewalk detection


In [10]:
model = Sequential()

#1st Layer
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(MaxPooling2D(pool_size=2))


#Upsampling Part
model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))


model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

model.add(UpSampling2D(size=(2, 2), data_format=None, interpolation='nearest'))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(1080, 1920, 1)))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001))

model.add(Dense(3, activation='softmax'))

In [11]:
(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1078, 1918, 32)    320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 1078, 1918, 32)    128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1076, 1916, 32)    9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 1076, 1916, 32)    128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 538, 958, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 536, 956, 32)      9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 536, 956, 32)     

In [0]:
!pip install tensorflow